# Proyecto de Extracción de Datos de la API Open Weather y Creación de Tabla en Amazon Redshift por *Carlos Horta*

## *Introducción*

Este primer entregable, de un total de tres, tiene como consigna el desarrollo de un script que permita la extracción de datos desde una API pública y, simultáneamente, la creación de la tabla correspondiente en Amazon RedShift. La finalidad de este proyecto es establecer un punto de partida, un script ETL inicial, que será fundamental en la realización del proyecto final. El script en cuestión deberá ser capaz de extraer datos en formato JSON y procesarlos en un diccionario de Python. Además, se espera la creación de una versión preliminar de la tabla donde los datos extraídos se cargarán posteriormente.

## *API seleccionada*
Para llevar a cabo este proyecto, se ha optado por utilizar la API de Open Weather, específicamente para obtener datos relacionados con el pronóstico del tiempo en la Ciudad de México durante los próximos cinco días, con intervalos de actualización cada tres horas. Del archivo JSON proporcionado por la API, se focalizará la atención en la extracción de información de los apartados 'main' y 'weather', excluyendo otros detalles como 'clouds', 'wind', 'visibility' y 'sys'.

-----

In [1]:
# Instalación de bibliotecas necesarias para el proyecto

%pip install pandas
%pip install requests
%pip install sqlalchemy
%pip install "redshift_connector[full]" sqlalchemy-redshift
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importar las bibliotecas necesarias

from configparser import ConfigParser
import pandas as pd
import psycopg2
import sqlalchemy as sa
from pathlib import Path
import requests
from sqlalchemy.exc import SQLAlchemyError

In [3]:
# Crear una instancia de ConfigParser
config = ConfigParser()

# Leer el archivo de configuración
config_dir = "config/config.ini"
config.read(config_dir)  

# Acceder a la sección 'redshift'
redshift_username = config['redshift']['username']
redshift_password = config['redshift']['password']
redshift_host = config['redshift']['host']
redshift_port = config['redshift']['port']
redshift_database = config['redshift']['database']

# Acceder a la sección 'api_openweather'
openweather_name = config['api_openweather']['name']
openweather_key = config['api_openweather']['key']

In [4]:
# Realizar la solicitud HTTP
base_url = "https://api.openweathermap.org/data/2.5/"
endpoint = f"forecast?lat=19.42847&lon=-99.12766&units=metric&appid={openweather_key}"
url = f"{base_url}{endpoint}"
resp = requests.get(url)

# Verificar si la solicitud se realizó con éxito
if resp.status_code == 200:
    data = resp.json()  # Convertir la respuesta JSON en un diccionario de Python

    if data:
        pronosticos = data.get('list', [])

        if pronosticos:
            # Crear un DataFrame de Pandas con los datos de interés
            df = pd.DataFrame(pronosticos)[['dt_txt', 'main', 'weather']]

            # Transformar los datos JSON en las columnas deseadas
            df['temp'] = df['main'].apply(lambda x: x['temp'])
            df['feels_like'] = df['main'].apply(lambda x: x['feels_like'])
            df['temp_min'] = df['main'].apply(lambda x: x['temp_min'])
            df['temp_max'] = df['main'].apply(lambda x: x['temp_max'])
            df['pressure'] = df['main'].apply(lambda x: x['pressure'])
            df['sea_level'] = df['main'].apply(lambda x: x['sea_level'])
            df['grnd_level'] = df['main'].apply(lambda x: x['grnd_level'])
            df['humidity'] = df['main'].apply(lambda x: x['humidity'])

            df['weather_id'] = df['weather'].apply(lambda x: x[0]['id'])
            df['weather_main'] = df['weather'].apply(lambda x: x[0]['main'])
            df['weather_description'] = df['weather'].apply(lambda x: x[0]['description'])

            # Eliminar las columnas JSON que ya no son necesarias
            df = df.drop(columns=['main', 'weather'])

            # Configuración de la conexión a Amazon Redshift desde config.ini
            engine = create_engine(f'redshift+psycopg2://{redshift_username}:{redshift_password}@{redshift_host}:{redshift_port}/{redshift_database}')

 # Definir la estructura de la tabla y crear la tabla en Amazon Redshift
            table_name = 'weather_data'
            create_table_sql = """
            CREATE TABLE IF NOT EXISTS weather_data (
                dt_txt TIMESTAMP PRIMARY KEY,
                temp FLOAT,
                feels_like FLOAT,
                temp_min FLOAT,
                temp_max FLOAT,
                pressure INT,
                sea_level INT,
                grnd_level INT,
                humidity INT,
                weather_main VARCHAR(255),
                weather_description VARCHAR(255)
            ) DISTSTYLE EVEN;
            """
            engine.execute(create_table_sql)

            try:
                # Cargar los datos en Amazon Redshift
                df.to_sql(table_name, engine, if_exists='append', index=False)
                print('Datos cargados exitosamente en Amazon Redshift.')
            except SQLAlchemyError as e:
                print(f'Error al cargar datos en Amazon Redshift: {str(e)}')
        else:
            print('No se encontraron pronósticos para la Ciudad de México en la respuesta de la API.')
    else:
        print('No se pudo obtener una respuesta válida de la API de OpenWeatherMap.')
else:
    print(f'Error al realizar la solicitud HTTP. Código de estado: {resp.status_code}')


C:\Users\carlo\AppData\Local\Temp\ipykernel_43668\4181528118.py:55: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute(create_table_sql)


Datos cargados exitosamente en Amazon Redshift.


----

## *Conclusión*

En este primer entregable del proyecto, hemos logrado desarrollar un script ETL inicial que nos permite extraer datos de una API pública y, al mismo tiempo, crear la estructura de la tabla correspondiente en Amazon RedShift. El objetivo de este ejercicio es sentar las bases para el proyecto final, donde se realizará un proceso ETL más completo y se explotarán los datos de manera más profunda.

Hemos seleccionado la API de Open Weather para obtener datos precisos sobre el pronóstico del tiempo en la Ciudad de México durante los próximos cinco días, con actualizaciones cada tres horas. El script se enfoca en la extracción de información relevante de los apartados 'main' y 'weather' del archivo JSON proporcionado por la API, dejando de lado otros detalles que no son necesarios para este proyecto.

A medida que avanzamos en el proyecto, este script inicial se convertirá en una parte esencial de nuestro flujo de trabajo de procesamiento de datos. La extracción y transformación de datos se volverán más sofisticadas, y la carga en la base de datos se optimizará para una explotación efectiva de la información. En resumen, este entregable marca el inicio de un proyecto emocionante en el que utilizaremos nuestras habilidades técnicas para obtener insights valiosos de los datos.